<a href="https://colab.research.google.com/github/taylorec/Design-Patterns-with-Python/blob/main/4)_Behavioral_Design_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Behavioral patterns deal with object interconnection and algorithms.

## The Chain of Responsibility pattern

The Chain of Responsibility pattern offers an elegant way to handle requests by passing them through a chain of handlers. Each handler in the chain has the autonomy to decide whether it can process the request or if it should delegate it further along the chain. This pattern shines when dealing with operations that involve multiple handlers but don’t necessarily require all of them to be involved.

The purpose of the chain of responsibility design pattern involves senders and receivers.
Specifically, the chain of responsibility design pattern calls for the decoupling of the
sender and receiver. Objects can be sent to a series of receivers without the sender being
concerned about which receiver handles the request. The request is sent along a chain of receivers and
only one of them will process the request.

In [ ]:
class Event:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

In [ ]:
class Widget:
    def __init__(self, parent=None):
        self.parent = parent

    def handle(self, event):
        handler = 'handle_{event}'.format(event=event)
        if hasattr(self, handler):
            method = getattr(self, handler)
            method(event)
        elif self.parent is not None:
            self.parent.handle(event)
        elif hasattr(self, 'handle_default'):
            self.handle_default(event)

In [ ]:
class MainWindow(Widget):
    def handle_close(self, event):
        print('MainWindow: {event}'.format(event=event))

    def handle_default(self, event):
        print('MainWindow Default: {event}'.format(event=event))

In [ ]:
class SendDialog(Widget):
    def handle_paint(self, event):
        print('SendDialog: {event}'.format(event=event))

In [ ]:
class MsgText(Widget):
    def handle_down(self, event):
        print('MsgText: {event}'.format(event=event))

In [ ]:
def main():
    mw = MainWindow()
    sd = SendDialog(mw)
    msg = MsgText(sd)

    for e in ('down', 'paint', 'unhandled', 'close'):
        evt = Event(e)
        print('Sending event -{evt}- to MainWindow'.format(evt=evt))
        mw.handle(evt)
        print('Sending event -{evt}- to SendDialog'.format(evt=evt))
        sd.handle(evt)
        print('Sending event -{evt}- to MsgText'.format(evt=evt))
        msg.handle(evt)

In [ ]:
if __name__ == '__main__':
    main()

Sending event -down- to MainWindow
MainWindow Default: down
Sending event -down- to SendDialog
MainWindow Default: down
Sending event -down- to MsgText
MsgText: down
Sending event -paint- to MainWindow
MainWindow Default: paint
Sending event -paint- to SendDialog
SendDialog: paint
Sending event -paint- to MsgText
SendDialog: paint
Sending event -unhandled- to MainWindow
MainWindow Default: unhandled
Sending event -unhandled- to SendDialog
MainWindow Default: unhandled
Sending event -unhandled- to MsgText
MainWindow Default: unhandled
Sending event -close- to MainWindow
MainWindow: close
Sending event -close- to SendDialog
MainWindow: close
Sending event -close- to MsgText
MainWindow: close


## The Command pattern

The purpose of the command design pattern is to send requests as objects.
This pattern, also referred to as the transaction or action design pattern,
permits the sending of requests without knowing any details about the receiver
or even about what is being requested.

In [ ]:
import os

In [ ]:
verbose = True

In [ ]:
class RenameFile:

    def __init__(self, src, dest):
        self.src = src
        self.dest = dest

    def execute(self):
        if verbose:
            print(f"[renaming '{self.src}' to '{self.dest}']")
        os.rename(self.src, self.dest)

    def undo(self):
        if verbose:
            print(f"[renaming '{self.dest}' back to '{self.src}']")
        os.rename(self.dest, self.src)

In [ ]:
class CreateFile:

    def __init__(self, path, txt='hello world\n'):
        self.path = path
        self.txt = txt

    def execute(self):
        if verbose:
            print(f"[creating file '{self.path}']")
        with open(self.path, mode='w', encoding='utf-8') as out_file:
            out_file.write(self.txt)

    def undo(self):
        delete_file(self.path)

In [ ]:
class ReadFile:

    def __init__(self, path):
        self.path = path

    def execute(self):
        if verbose:
            print(f"[reading file '{self.path}']")
        with open(self.path, mode='r', encoding='utf-8') as in_file:
            print(in_file.read(), end='')

In [ ]:
def delete_file(path):
    if verbose:
        print(f"deleting file {path}")
    os.remove(path)

In [ ]:
def main():

    orig_name, new_name = 'file1', 'file2'

    commands = (CreateFile(orig_name),
            ReadFile(orig_name),
            RenameFile(orig_name, new_name))

    [c.execute() for c in commands]

    answer = input('reverse the executed commands? [y/n] ')

    if answer not in 'yY':
        print(f"the result is {new_name}")
        exit()

    for c in reversed(commands):
        try:
            c.undo()
        except AttributeError as e:
            print("Error", str(e))

In [ ]:
if __name__ == "__main__":
    main()

[creating file 'file1']
[reading file 'file1']
hello world
[renaming 'file1' to 'file2']
reverse the executed commands? [y/n] y
[renaming 'file2' back to 'file1']
Error 'ReadFile' object has no attribute 'undo'
deleting file file1


## The Observer pattern

The Observer pattern describes a publish-subscribe relationship between a single object, the publisher, which is also known as the subject or observable, and one or more objects, the subscribers, also known as observers. So, the subject notifies the subscribers of any state changes, typically by calling one of their methods.

The observer design pattern requires a one-to-many object dependency.
The purpose of the dependency is to update subscriber objects when a change
is made to the publisher object's state.

In [ ]:
class Publisher:
    def __init__(self):
        self.observers = []

    def add(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
        else:
            print(f'Failed to add: {observer}')

    def remove(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print(f'Failed to remove: {observer}')

    def notify(self):
        [o.notify(self) for o in self.observers]

In [ ]:
class DefaultFormatter(Publisher):
    def __init__(self, name):
        Publisher.__init__(self)
        self.name = name
        self._data = 0

    def __str__(self):
        return f"{type(self).__name__}: '{self.name}' has data = {self._data}"

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_value):
        try:
            self._data = int(new_value)
        except ValueError as e:
            print(f'Error: {e}')
        else:
            self.notify()

In [ ]:
class HexFormatterObs:
    def notify(self, publisher):
        value = hex(publisher.data)
        print(f"{type(self).__name__}: '{publisher.name}' has now hex data = {value}")

In [ ]:
class BinaryFormatterObs:
    def notify(self, publisher):
        value = bin(publisher.data)
        print(f"{type(self).__name__}: '{publisher.name}' has now bin data = {value}")

In [ ]:
def main():
    df = DefaultFormatter('test1')
    print(df)

    print()
    hf = HexFormatterObs()
    df.add(hf)
    df.data = 3
    print(df)

    print()
    bf = BinaryFormatterObs()
    df.add(bf)
    df.data = 21
    print(df)

    print()
    df.remove(hf)
    df.data = 40
    print(df)

    print()
    df.remove(hf)
    df.add(bf)

    df.data = 'hello'
    print(df)

    print()
    df.data = 15.8
    print(df)

In [ ]:
if __name__ == '__main__':
    main()

DefaultFormatter: 'test1' has data = 0

HexFormatterObs: 'test1' has now hex data = 0x3
DefaultFormatter: 'test1' has data = 3

HexFormatterObs: 'test1' has now hex data = 0x15
BinaryFormatterObs: 'test1' has now bin data = 0b10101
DefaultFormatter: 'test1' has data = 21

BinaryFormatterObs: 'test1' has now bin data = 0b101000
DefaultFormatter: 'test1' has data = 40

Failed to remove: <__main__.HexFormatterObs object at 0x7e45d7fdf010>
Failed to add: <__main__.BinaryFormatterObs object at 0x7e45d7fdff40>
Error: invalid literal for int() with base 10: 'hello'
DefaultFormatter: 'test1' has data = 40

BinaryFormatterObs: 'test1' has now bin data = 0b1111
DefaultFormatter: 'test1' has data = 15


## The Interpreter pattern

The interpreter design pattern is used to establish a grammatical representation
and an interpreter that interprets language.

In [ ]:
from pyparsing import Word, OneOrMore, Optional, Group, Suppress, alphanums

In [ ]:
class Gate:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return 'open' if self.is_open else 'closed'

    def open(self):
        print('opening the gate')
        self.is_open = True

    def close(self):
        print('closing the gate')
        self.is_open = False

In [ ]:
class Garage:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return 'open' if self.is_open else 'closed'

    def open(self):
        print('opening the garage')
        self.is_open = True

    def close(self):
        print('closing the garage')
        self.is_open = False

In [ ]:
class Aircondition:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return 'on' if self.is_on else 'off'

    def turn_on(self):
        print('turning on the air condition')
        self.is_on = True

    def turn_off(self):
        print('turning off the air condition')
        self.is_on = False

In [ ]:
class Heating:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return 'on' if self.is_on else 'off'

    def turn_on(self):
        print('turning on the heating')
        self.is_on = True

    def turn_off(self):
        print('turning off the heating')
        self.is_on = False

In [ ]:
class Boiler:
    def __init__(self):
        self.temperature = 83

    def __str__(self):
        return f'boiler temperature: {self.temperature}'

    def increase_temperature(self, amount):
        print(f"increasing the boiler's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the boiler's temperature by {amount} degrees")
        self.temperature -= amount

In [ ]:
class Fridge:
    def __init__(self):
        self.temperature = 34

    def __str__(self):
        return f'fridge temperature: {self.temperature}'

    def increase_temperature(self, amount):
        print(f"increasing the fridge's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the fridge's temperature by {amount} degrees")
        self.temperature -= amount

In [ ]:
word = Word(alphanums)
command = Group(OneOrMore(word))
command

Group:({W:(0-9A-Za-z)}...)

In [ ]:
def main():
    word = Word(alphanums)
    command = Group(OneOrMore(word))
    token = Suppress("->")
    device = Group(OneOrMore(word))
    argument = Group(OneOrMore(word))
    event = command + token + device + Optional(token + argument)

    gate = Gate()
    garage = Garage()
    airco = Aircondition()
    heating = Heating()
    boiler = Boiler()
    fridge = Fridge()

    tests = ('open -> gate',
             'close -> garage',
             'turn on -> air condition',
             'turn off -> heating',
             'increase -> boiler temperature -> 5 degrees',
             'decrease -> fridge temperature -> 2 degrees')

    open_actions = {'gate':gate.open,
                    'garage':garage.open,
                    'air condition':airco.turn_on,
                    'heating':heating.turn_on,
                    'boiler temperature':boiler.increase_temperature,
                    'fridge temperature':fridge.increase_temperature}
    close_actions = {'gate':gate.close,
                     'garage':garage.close,
                     'air condition':airco.turn_off,
                     'heating':heating.turn_off,
                     'boiler temperature':boiler.decrease_temperature,
                     'fridge temperature':fridge.decrease_temperature}

    for t in tests:
        if len(event.parseString(t)) == 2: # no argument
            cmd, dev = event.parseString(t)
            cmd_str, dev_str = ' '.join(cmd), ' '.join(dev)
            if 'open' in cmd_str or 'turn on' in cmd_str:
                open_actions[dev_str]()
            elif 'close' in cmd_str or 'turn off' in cmd_str:
                close_actions[dev_str]()
        elif len(event.parseString(t)) == 3: # argument
            cmd, dev, arg = event.parseString(t)
            cmd_str = ' '.join(cmd)
            dev_str = ' '.join(dev)
            arg_str = ' '.join(cmd)
            num_arg = 0
            try:
                # extract the numeric part
                num_arg = int(arg_str.split()[0])
            except ValueError as err:
                print(f"expected number but got: '{arg_str[0]}'")
            if 'increase' in cmd_str and num_arg > 0:
                open_actions[dev_str](num_arg)
            elif 'decrease' in cmd_str and num_arg > 0:
                close_actions[dev_str](num_arg)

    print('The gate is open:', gate.is_open)
    print('The garage is open:', garage.is_open)
    print('The air condition is on:', airco.is_on)
    print('The heating is on:', heating.is_on)
    print('The boiler temperature is ', boiler.temperature)
    print('The fridge temperature is ', fridge.temperature)


In [ ]:
if __name__ == '__main__':
    main()

opening the gate
closing the garage
turning on the air condition
turning off the heating
expected number but got: 'i'
expected number but got: 'd'
The gate is open: True
The garage is open: False
The air condition is on: True
The heating is on: False
The boiler temperature is  83
The fridge temperature is  34


## The Memento pattern

The memento design pattern saves an object's current internal state as a
memento so that it can be referred to and restored to.

In [ ]:
import pickle

In [ ]:
class Quote:

    def __init__(self, text, author):
        self.text = text
        self.author = author

    def save_state(self):
        current_state = pickle.dumps(self.__dict__)

        return current_state

    def restore_state(self, memento):
        previous_state = pickle.loads(memento)

        self.__dict__.clear()
        self.__dict__.update(previous_state)

    def __str__(self):
        return f'{self.text} - By {self.author}.'

In [ ]:
def main():
    print('Quote 1')
    q1 = Quote("A room without books is like a body without a soul.",
               'Unknown author')
    print(f'\nOriginal version:\n{q1}')
    q1_mem = q1.save_state()

    # Now, we found the author's name
    q1.author = 'Marcus Tullius Cicero'
    print(f'\nWe found the author, and did an update:\n{q1}')

    # Restoring previous state (Undo)
    q1.restore_state(q1_mem)
    print(f'\nWe had to restore the previous version:\n{q1}')

    print()
    print('Quote 2')
    q2 = Quote("To be you in a world that is constantly trying to make you be something else is the greatest accomplishment.",
               'Ralph Waldo Emerson')
    print(f'\nOriginal version:\n{q2}')
    q2_mem1 = q2.save_state()

    # changes to the text
    q2.text = "To be yourself in a world that is constantly trying to make you something else is the greatest accomplishment."
    print(f'\nWe fixed the text:\n{q2}')
    q2_mem2 = q2.save_state()

    q2.text = "To be yourself when the world is constantly trying to make you something else is the greatest accomplishment."
    print(f'\nWe fixed the text again:\n{q2}')

    # Restoring previous state (Undo)
    q2.restore_state(q2_mem2)
    print(f'\nWe had to restore the 2nd version, the correct one:\n{q2}')

In [ ]:
if __name__ == "__main__":
    main()

Quote 1

Original version:
A room without books is like a body without a soul. - By Unknown author.

We found the author, and did an update:
A room without books is like a body without a soul. - By Marcus Tullius Cicero.

We had to restore the previous version:
A room without books is like a body without a soul. - By Unknown author.

Quote 2

Original version:
To be you in a world that is constantly trying to make you be something else is the greatest accomplishment. - By Ralph Waldo Emerson.

We fixed the text:
To be yourself in a world that is constantly trying to make you something else is the greatest accomplishment. - By Ralph Waldo Emerson.

We fixed the text again:
To be yourself when the world is constantly trying to make you something else is the greatest accomplishment. - By Ralph Waldo Emerson.

We had to restore the 2nd version, the correct one:
To be yourself in a world that is constantly trying to make you something else is the greatest accomplishment. - By Ralph Waldo Em

## The Iterator pattern

The purpose of the iterator design pattern is to grant access to an
object's members without sharing the encapsulated data structures.

In [ ]:
class FootballTeamIterator:

    def __init__(self, members):
        # the list of players and coaches
        self.members = members
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.members):
            val = self.members[self.index]
            self.index += 1
            return val

In [ ]:
class FootballTeam:

    def __init__(self, members):
        self.members = members

    def __iter__(self):
        return FootballTeamIterator(self.members)

In [ ]:
def main():
    members = [f'player{str(x)}' for x in range(1, 23)]
    members = members + ['coach1', 'coach2', 'coach3']
    team = FootballTeam(members)
    team_it = iter(team)
    for i in range(len(members)):
      print(next(team_it))

In [ ]:
if __name__ == '__main__':
    main()

player1
player2
player3
player4
player5
player6
player7
player8
player9
player10
player11
player12
player13
player14
player15
player16
player17
player18
player19
player20
player21
player22
coach1
coach2
coach3
